<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br><b>Prototype Target Dashboard</b> <br>
Contact author: Douglas Tucker <br>
Last verified to run: 2024-10-10 <br>
LSST Science Pipelines version: Weekly 2024_37 <br>
Container size: medium <br>

**Description:** A prototype dashboard for exploring targets (and science programs) for a given night.  This version focuses on AuxTel observations, but should be also useful for ComCom and LSSTcam commissioning, when there will be a greater diversity of observational programs than those provided by the survey scheduler during standard operations.

See Jira issue <a href=https://rubinobs.atlassian.net/browse/SP-924>SP-924</a>.

**Credit:** Based heavily on the <a href=https://github.com/sylvielsstfr/AuxTelComm/blob/main/notebooks_usdf/ana_auxtelprod_jn/spectractor_v3.1.0_May2024/ListOfExposures-hologram-oga.ipynb> List of Exposures at USDF in OGA</a> notebook by Sylvie Dagoret-Campagne.  This initial "Prototype Target Dashboard" notebook would not have been possible without it.

In [ ]:

## 1. Preliminaries
#
### 1.1 Parameter input
#
### LSSTComSamSim for OR4:
##instrument = 'LSSTComCamSim'
##repo = 'embargo_or4'
##day_obs = '2024-06-25'
##collection = ['LSSTComCamSim/prompt/output-2024-06-25',
##              'LSSTComCamSim/prompt/output-2024-06-26',
##              'LSSTComCamSim/prompt/output-2024-06-27']
##collection_sky = ['LSSTComCamSim/runs/nightlyValidation/20240625/w_2024_25/DM-44966',
##                  'LSSTComCamSim/runs/nightlyValidation/20240626/w_2024_25/DM-44966',
##                  'LSSTComCamSim/runs/nightlyValidation/20240627/w_2024_25/DM-44966']
##skymap_name = 'ops_rehersal_prep_2k_v1'
##
##col_sciprog = 'science_program'
###col_target = 'target'
###col_target = 'HPX8'  # HPX8 is used as a proxy for target_name or block
##col_target = 'field_name'  # field_name is used as a proxy for target_name or block
##col_filter = 'filter'
##col_id = 'id'
#
#
## LATISS:
#instrument = 'LATISS'
#repo = '/repo/embargo'
#day_obs = '2024-08-07'
#collection = ['LATISS/prompt/output-2024-08-07']
#collection_sky = ['LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240513/w_2024_20/PREOPS-5146']
#skymap_name = 'latiss_v1'
#
#col_sciprog = 'science_program'
#col_target = 'target'
##col_target = 'HPX8'  # HPX8 is used as a proxy for target_name or block
##col_target = 'field_name'  # field_name is used as a proxy for target_name or block
#col_filter = 'filter'
#col_id = 'id'



In [ ]:
from datetime import datetime

# Convert the string to a datetime object
day_obs_obj = datetime.strptime(day_obs, '%Y-%m-%d')

# Format the datetime object as an integer
day_obs_int = int(day_obs_obj.strftime('%Y%m%d'))

print(day_obs_int)  # Output: 20240807

day_obs = day_obs_int

In [ ]:

# Set instrument, day_obs, and (sometimes) pipeline:

# LSSTComSamSim for OR4:
#instrument = 'LSSTComCamSim'
#day_obs = 20240625
##pipeline = 'nv' # either 'nv' (nightly validation), 'pp' (prompt processing), or 'drp' (data release pipeline)
#pipeline = 'pp' # either 'nv' (nightly validation), 'pp' (prompt processing), or 'drp' (data release pipeline)
##pipeline = 'drp' # either 'nv' (nightly validation), 'pp' (prompt processing), or 'drp' (data release pipeline)


## LATISS:
#instrument = 'LATISS'
#day_obs = 20240807

#pipeline = 'pp' # either 'pp' (prompt processing) or 'drp' (data release pipeline)
##pipeline = 'drp' # either 'pp' (prompt processing) or 'drp' (data release pipeline)


In [ ]:

## Set repo, collection, and columns to use based on instrument, day_obs, and (sometimes) pipeline:
#
#if instrument == 'LSSTComCamSim':
#
#    # From Meredith Rawls:
#    repo = 'embargo_or4'
#    instrument = 'LSSTComCamSim'
#    skymap_name = 'ops_rehersal_prep_2k_v1'
#    
#    nv_collections = ['LSSTComCamSim/runs/nightlyValidation/20240625/w_2024_25/DM-44966',
#                      'LSSTComCamSim/runs/nightlyValidation/20240626/w_2024_25/DM-44966',
#                      'LSSTComCamSim/runs/nightlyValidation/20240627/w_2024_25/DM-44966']
#    
#    drp_collections = 'LSSTComCamSim/runs/DRP/OR4/w_2024_25/DM-45066'
#    
#    pp_collections = ['LSSTComCamSim/prompt/output-2024-06-25',
#                      'LSSTComCamSim/prompt/output-2024-06-26',
#                      'LSSTComCamSim/prompt/output-2024-06-27']
#    
#    if pipeline == 'drp':
#        collection=drp_collections
#        collection_sky=collection
#    elif pipeline == 'nv':
#        collection=nv_collections
#        collection_sky=nv_collections
#    elif pipeline == 'pp': 
#        collection=pp_collections
#        # prompt processing does not seem to have its own sky map?
#        #  use the nv_collections skymap instead?
#        collection_sky=nv_collections
#    else:
#        print('ERROR:  unrecognized pipeline.  Stopping here...')
#        raise StopExecution
#
#    col_sciprog = 'science_program'
#    #col_target = 'target'
#    #col_target = 'HPX8'  # HPX8 is used as a proxy for target_name or block
#    col_target = 'field_name'  # field_name is used as a proxy for target_name or block
#    col_filter = 'filter'
#    col_id = 'id'
#
#elif instrument == 'LATISS':
#
#    skymap_name = 'latiss_v1'
#
## From Huan Lin:
#    drp_repo = 'embargo_or4'
#    drp_collections = ['LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240807/w_2024_30/PREOPS-5352']
#    drp_collections_sky = drp_collections
#
#    pp_repo = '/repo/embargo'
#    pp_collections = ['LATISS/prompt/output-2024-08-07']
#    pp_collections_sky = ['LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240513/w_2024_20/PREOPS-5146']
#    
#    if pipeline == 'drp':
#        repo=drp_repo
#        collection=drp_collections
#        collection_sky=collection
#    elif pipeline == 'pp': 
#        repo=pp_repo
#        collection=pp_collections
#        # prompt processing does not seem to have its own sky map?
#        collection_sky=pp_collections_sky
#    else:
#        print('ERROR:  unrecognized pipeline.  Stopping here...')
#        raise StopExecution
#
#    col_sciprog = 'science_program'
#    col_target = 'target'
#    #col_target = 'HPX8'  # HPX8 is used as a proxy for target_name or block
#    #col_target = 'field_name'  # field_name is used as a proxy for target_name or block
#    col_filter = 'filter'
#    col_id = 'id'
# 
#else:
#
#    print('Unsupported instrument.  Stopping here...')
#    raise StopExecution
#    

In [ ]:

## 1.2 Import packages

In [ ]:

import sys
import os
import io
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import healpy as hp
from matplotlib.colors import LogNorm
from IPython.display import display, HTML
import lsst.afw.display as afwDisplay
import lsst.daf.butler as dafButler
import lsst.geom as geom

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

import warnings

In [ ]:

## 1.3 Define functions and parameters

In [ ]:

# Create a function to create a pandas DataFrame, df_grouped, that contains the results of performing a 2D groupby 
# operation on a pandas DataFrame, df, and counts the number of entries in the col_count column for each (col_x, col_y).  
# One can choose to include a count of zero for cases where (col_x, col_y) have no entries in col_count (the default), 
# or to exclude those cases.  The resulting pandas DataFrame, df_grouped, is also sorted by (col_x, col_y).

# (Thanks to the ChatGPT on Poe.com and to https://stackoverflow.com/questions/37003100/pandas-groupby-for-zero-values 
# for help in creating this function!)

def df2Dgroupby(df, col_x, col_y, col_count, fill_zeros=True):

    if fill_zeros==True:

        df_grouped = (
            df.groupby([col_x, col_y])[col_count]
            .count()
            .unstack(fill_value=0)
            .stack()
            .reset_index()
            .rename(columns={0: "number"})
            .sort_values([col_x, col_y])
        )

    else:

        df_grouped = (
            df.groupby([col_x, col_y])[col_count]
            .count()
            .reset_index()
            .rename(columns={col_count: "number"})
            .sort_values([col_x, col_y])
        )
        
    return df_grouped

In [ ]:

# The same, but for performing the `groupby` over 3 columns. 

# (Thanks to the ChatGPT on Poe.com and to https://stackoverflow.com/questions/37003100/pandas-groupby-for-zero-values 
# for help in creating this function!)

def df3Dgroupby(df, col_x, col_y, col_z, col_count, fill_zeros=True):

    if fill_zeros==True:

        df_grouped = (
            df.groupby([col_x, col_y, col_z])[col_count]
            .count()
            .unstack(fill_value=0)
            .stack()
            .reset_index()
            .rename(columns={0: "number"})
            .sort_values([col_x, col_y, col_z])
        )

    else:

        df_grouped = (
            df.groupby([col_x, col_y, col_z])[col_count]
            .count()
            .reset_index()
            .rename(columns={col_count: "number"})
            .sort_values([col_x, col_y, col_z])
        )
        
    return df_grouped

In [ ]:

# The same, but for performing the `groupby` over 4 columns. 

# (Thanks to the ChatGPT on Poe.com and to https://stackoverflow.com/questions/37003100/pandas-groupby-for-zero-values 
# for help in creating this function!)

def df4Dgroupby(df, col_w, col_x, col_y, col_z, col_count, fill_zeros=True):

    if fill_zeros==True:

        df_grouped = (
            df.groupby([col_w, col_x, col_y, col_z])[col_count]
            .count()
            .unstack(fill_value=0)
            .stack()
            .reset_index()
            .rename(columns={0: "number"})
            .sort_values([col_w, col_x, col_y, col_z])
        )

    else:

        df_grouped = (
            df.groupby([col_w, col_x, col_y, col_z])[col_count]
            .count()
            .reset_index()
            .rename(columns={col_count: "number"})
            .sort_values([col_w, col_x, col_y, col_z])
        )
        
    return df_grouped

In [ ]:

# Create a function to perform a many-to-one match based on RA, DEC sky coordinates:
# (Kudos to Claude-3.5-Sonnet AI from Poe.com for this function!)

def skyMatchCatManytoOne(df_a, df_a_ra_colname, df_a_dec_colname, df_b, df_b_ra_colname, df_b_dec_colname, max_sep_deg, verboseFlag=False):

    from astropy.coordinates import SkyCoord # High-level coordinates
    from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
    from astropy.coordinates import Angle, Latitude, Longitude  # Angles
    from astropy import units as u
    from astropy.coordinates.matching import search_around_sky
    import numpy as np
    import pandas as pd

    # Create SkyCoord objects for both DataFrames
    coords_a = SkyCoord(ra=df_a[df_a_ra_colname].values*u.degree, dec=df_a[df_a_dec_colname].values*u.degree)
    coords_b = SkyCoord(ra=df_b[df_b_ra_colname].values*u.degree, dec=df_b[df_b_dec_colname].values*u.degree)

    # Set a maximum separation for matching (adjust as needed)
    max_sep_deg = max_sep_deg * u.degree

    # Perform the search
    idx_a, idx_b, d2d, _ = search_around_sky(coords_a, coords_b, max_sep_deg)

    # Create a DataFrame with the matches
    df_matches = pd.DataFrame({
        'idx_a': idx_a,
        'idx_b': idx_b,
        'separation': d2d.degree
    })

    # Group by idx_a and find the minimum separation for each
    best_matches = df_matches.loc[df_matches.groupby('idx_a')['separation'].idxmin()]

    # Create the final matched DataFrame, keeping all entries from df_a
    df_matched = df_a.copy()

    # Prepare the data from df_b to be joined
    df_b_matched = df_b.iloc[best_matches['idx_b']].copy()
    df_b_matched['idx_a'] = best_matches['idx_a'].values
    df_b_matched['separation'] = best_matches['separation'].values
    df_b_matched = df_b_matched.set_index('idx_a')

    # Perform the left join
    df_matched = df_matched.join(df_b_matched, rsuffix='_b')
    
    # Reset index if needed
    df_matched = df_matched.reset_index(drop=True)

    # Fill NaN values in the 'separation' column with a sentinel value (e.g., -1) to indicate no match
    df_matched['separation'] = df_matched['separation'].fillna(-1)

    # Optionally, you can add a boolean column to easily identify matched and unmatched rows
    df_matched['has_match'] = df_matched['separation'] != -1

    if verboseFlag:
        # Print some statistics
        print(f"Total entries in df_a: {len(df_a)}")
        print(f"Entries in df_a with matches: {df_matched['has_match'].sum()}")
        print(f"Entries in df_a without matches: {(~df_matched['has_match']).sum()}")

    return df_matched

In [ ]:

# Define some healpix tools <span style="color:red; font-weight:bold">(Maybe no longer necessary?)</span>:
# (Thanks to Sahar Allam and Alex Drlica-Wagner!)

##################################
def radec2thetaphi(ra, dec):
    import numpy as np
    return (90-dec)*np.pi/180., ra*np.pi/180.

##################################
#DESDM uses nside=128, nest=True
#Alex Drlica Wagner's healpixelated Gaia DR2 on des40 uses nside=32, nest=False
def getipix(nside,ra,dec,nest=True):
    import healpy as hp
    theta, phi = radec2thetaphi(ra, dec)
    ipix = hp.pixelfunc.ang2pix(nside, theta, phi, nest)
    return ipix

##################################

In [ ]:

# Define a function to return the tract id, given the ra, dec and a skymap:

def find_tract(ra,dec,skymap):

    import lsst.geom as geom
    
    try:
        sky_point = geom.SpherePoint(ra * geom.degrees,
                                     dec * geom.degrees)

        tract = skymap.findTract(sky_point)
        tract = tract.tract_id
    
        del sky_point

    except:
        
        tract = -9999
    
    return tract

In [ ]:

# Use queryDataIds to find tracts that have overlap with an exposure.
#  (Via Krzysz Findeisen and Meredith Rawls!  Plus a little help from the Poe.com AI Assistant.)

def find_tracts_overlapping_exposure(exposure_id, instrument, butler):
    
    tract_set = set(list(butler.registry.queryDataIds(dimensions=('visit', 'tract'), 
                                                      instrument=instrument, 
                                                      exposure=exposure_id)))
    
    tract_values = [item['tract'] for item in tract_set]
    tract_values = list(set(tract_values))
    # Note:  tract_values.sort() sorts in place and returns `None`!
    #  So either use `tract_values.sort()` or `tract_values = sorted(tract_values)`
    tract_values = sorted(tract_values)
    
    return tract_values

In [ ]:

# Return a string of the list of tracts overlapping the exposure.
#  If the function find_tracts_overlapping_exposure does not find a list,
#  use find_tract to find the tract nearest the center of the exposure.
#  If find_tract fails, it will return -9999.
#  Append an asterisk (*) to the tract list if it is provided by find_tract.

def return_string_of_tracts_overlapping_exposure_plus(exposure_id, instrument, butler, skymap, ra, dec):

    list_of_tracts = find_tracts_overlapping_exposure(exposure_id, instrument, butler)
    
    if not list_of_tracts:
        central_tract = find_tract(ra,dec,skymap)
        list_of_tracts_string = str(central_tract)+'*'
    else:
        list_of_tracts_string = ','.join(map(str, list_of_tracts))

    #print(exposure_id, list_of_tracts_string)

    return list_of_tracts_string


In [ ]:

# Define a class to stop "Run All" at a code cell containing the command "raise StopExecution":

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

In [ ]:

# Set default backend for afwDisplay to matplotlib:
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:

# Set a few parameters to use later, when plotting:

plt.style.use('tableau-colorblind10')

plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

In [ ]:

## 1.4 Add Ops Rehearsal 4 field table

In [ ]:

# This is a temporary fix to attach field names to the Ops Rehearsal 4 science exposures:
# Table of Ops Rehearsal 4 Fields from https://confluence.lsstcorp.org/pages/viewpage.action?pageId=259785406
ascii_table = """
Field                       RA          DEC     stellarDens     E_BV
Rubin_SV_095_-25           95.00      -25.00        2.92        0.05
Rubin_SV_125_-15          125.00      -15.00        3.78        0.06
DESI_SV3_R1               179.60        0.00        0.53        0.03
Rubin_SV_225_-40          225.00      -40.00       12.67        0.08
DEEP_A0                   216.00      -12.50        1.17        0.09
Rubin_SV_250_2            250.00        2.00        4.56        0.07
Rubin_SV_300_-41          300.00      -41.00        5.22        0.09
Rubin_SV_280_-48          280.00      -48.00       17.64        0.07
DEEP_B0                   310.00      -19.00        3.28        0.04
ELAIS_S1                    9.45      -44.00        0.56        0.01
XMM_LSS                    35.71       -4.75        0.47        0.03
ECDFS                      53.12      -28.10        0.61        0.01
COSMOS                    150.10        2.18        0.67        0.02
EDFS_A                     58.90      -49.31        0.75        0.01
EDFS_B                     63.60      -47.60        0.81        0.01"""

# Convert the string to a file-like object
ascii_io = io.StringIO(ascii_table)

# Read the ASCII table into a pandas DataFrame
df_field = pd.read_csv(ascii_io, delim_whitespace=True)

# Add healpix columns to this table, one very roughly the area of a 
# ComCam field-of-view (NSIDE=2^6=64 --> 0.84 sq deg) and one very roughly 
# the area of an LSSTCam field-of-view (NSIDE=2^4=16 --> 13 sq deg) 
# ***(Maybe no longer necessary?)***
df_field.loc[:,'HPX64'] = getipix(64, df_field.loc[:,'RA'].values, df_field.loc[:,'DEC'].values)
df_field.loc[:,'HPX16'] = getipix(16, df_field.loc[:,'RA'].values, df_field.loc[:,'DEC'].values)

## Display df_field
#df_field

In [ ]:

# 2. Access data for this repo, collection, and day of observation

In [ ]:

## 2.1 Instantiate butler and create registry

In [ ]:

butler = dafButler.Butler(repo, collections=collection)
registry = butler.registry

In [ ]:

## 2.2 Read in information from the `exposure` dimension and create pandas DataFrame

## Total Number of Exposures Returned from Butler Query for the Instrument and Night

In [ ]:

# Query the metadata for the `exposure` dimension, limiting the results to this particular instrument and day of observation:
query="instrument='%s' AND day_obs=%d" % (instrument, day_obs)
results = registry.queryDimensionRecords('exposure',where=query)

In [ ]:

# Stop executing if there are no results returned:

n_results = results.count()

if n_results <= 0:
    raise StopExecution
else:
    print("""There are %d results returned from querying the butler for instrument %s on %d.""" % (n_results, instrument, day_obs))

In [ ]:

# Instantiate a pandas `DataFrame` with useful columns available in the `exposure` dimension:
df_exp = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num',
                                    'time_start','time_end' ,'type', 'reason', 
                                    'target','filter','zenith_angle',
                                    'expos','ra','dec','skyangle',
                                    'azimuth','zenith','science_program',
                                    'jd','mjd'])

In [ ]:

# Read the query results into the new pandas `DataFrame`:

for count, info in enumerate(results):
    
    
    try:
        df_exp.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, 
                                  pd.to_datetime(info.timespan.begin.to_string()),
                                  pd.to_datetime(info.timespan.end.to_string()),
                                  info.observation_type, info.observation_reason, info.target_name, 
                                  info.physical_filter, info.zenith_angle, 
                                  info.exposure_time,info.tracking_ra, info.tracking_dec, 
                                  info.sky_angle,info.azimuth ,info.zenith_angle, 
                                  info.science_program, info.timespan.begin.jd,info.timespan.begin.mjd ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exp.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num, 
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string), 
                                  info.observation_type, info.observation_reason, info.target_name, 
                                  info.physical_filter, info.zenith_angle, 
                                  info.exposure_time,info.tracking_ra, info.tracking_dec, 
                                  info.sky_angle,info.azimuth ,info.zenith_angle, 
                                  info.science_program, info_timespan_begin_jd, info_timespan_begin_mjd ]
 
    


In [ ]:

# Re-cast the `id`, `day_obs`, and `seq_num` rows as `int`'s:
df_exp = df_exp.astype({"id": int,'day_obs': int,'seq_num':int})

In [ ]:

# Replace `NaN`'s in the `ra` and `dec` columns with zero.  
# (`NaN`'s in `ra`, `dec` wreak havoc for the healpix tools defined in Section 1.2 above.) 
# ***(Maybe no longer necessary?)***

df_exp['ra'] = df_exp['ra'].fillna(0)
df_exp['dec'] = df_exp['dec'].fillna(0)

In [ ]:

# Add healpix info 
# ***(Maybe no longer necessary?)***:
df_exp.loc[:,'HPX64'] = getipix(64, df_exp.loc[:,'ra'].values, df_exp.loc[:,'dec'].values)
df_exp.loc[:,'HPX16'] = getipix(16, df_exp.loc[:,'ra'].values, df_exp.loc[:,'dec'].values)

In [ ]:

# To add the field_name to df_exp, perform a left join of df_field on df_exp
# *** (Note use of HPX16, but maybe that part is no longer necessary?)***:

# First, perform the merge...
# (Before we had the table of fields, we matched by HPX16 value.)
#df_merged = df_exp.merge(df_field, on='HPX16', how='left', suffixes=('', '_fld'))
df_merged = skyMatchCatManytoOne(df_exp, 'ra', 'dec', df_field, 'RA', 'DEC', 3.)

# Next, define a function to fill NaN values in 'Field' with 'HPX16' for unmatched exposures
def fill_field(row):
    if pd.isna(row['Field']):
        return 'HPX16_'+str(row['HPX16'])  # Convert to string if you want 'Field' to be string type
    return row['Field']

# Apply the function to fill NaN values
df_merged['Field'] = df_merged.apply(fill_field, axis=1)

# Remove extraneous columns from df_merged
#df_merged = df_merged.drop(['RA', 'DEC', 'stellarDens', 'E_BV', 'HPX64_fld', 'HPX16_fld'], axis=1)
df_merged = df_merged.drop(['RA', 'DEC', 'stellarDens', 'E_BV', 'HPX64_b', 'HPX16_b'], axis=1)

# Rename 'Field' column to 'field_name'
df_merged = df_merged.rename(columns={'Field': 'field_name'})

# Rename df_merged to df_exp
df_exp = df_merged

## Display updated df_exp
#df_exp

In [ ]:
#butler_sky = dafButler.Butler(repo, collections=collection_sky)
#registry_sky = butler_sky.registry

In [ ]:
#for dt in sorted(registry_sky.queryDatasetTypes('*sky*')):
#     print(dt)

In [ ]:
#dt_skymap = registry_sky.getDatasetType('skyMap')
#print(dt_skymap)

In [ ]:
#dt_skymap.storageClass

In [ ]:
#skymap = butler_sky.get('skyMap')

In [ ]:
#butler_sky.collections.query_info('LATISS/prompt/output-2024-08-07')

In [ ]:
## try weekly_2024_37

In [ ]:

# Add the tract id as a column in df_exp.  
# First, access the skymap, then add the tract to the df_exp DataFrame
# (this can take SEVERAL minutes!):

butler_sky = dafButler.Butler(repo, collections=collection_sky)
registry_sky = butler_sky.registry
skymap = butler_sky.get('skyMap', skymap=skymap_name)

df_exp['tract'] = df_exp.apply(lambda row: find_tract(row['ra'], row['dec'], skymap), axis=1)
#df_exp['tract']  = df_exp.apply(lambda row: return_string_of_tracts_overlapping_exposure_plus(row['id'], instrument, repo, collection_sky, skymap_name, row['ra'], row['dec']), axis=1)
#df_exp['tract']  = df_exp.apply(lambda row: return_string_of_tracts_overlapping_exposure_plus(row['id'], instrument, butler_sky, skymap, row['ra'], row['dec']), axis=1)

In [ ]:

# Create a `DataFrame` containing just the science exposures:
df_sci = df_exp[df_exp.type == 'science']

In [ ]:

# 3. Analyse results of the exposure query

In [ ]:

# For the rest of the notebook, we do not want to restrict the the number of rows displayed for
# pandas tabular information.  Therefore, let us turn off the pandas maximum rows setting.  
# We will turn it back on at the end of the notebook.

# Change the display.max_rows option
pd.set_option("display.max_rows", None)

In [ ]:

## 3.2 Counts

In [ ]:

### 3.2.1 All Exposures

In [ ]:

# Total counts
df_exp_grouped_type = df_exp.groupby("type").count()["id"].reset_index().rename(columns={'id': 'number'})
df_exp_grouped_type.rename(columns={'number': 'n_exp'}, inplace=True)
#df_exp_grouped_type

In [ ]:


# Group df_exp by 'type', col_filter.
df_exp_grouped = df2Dgroupby(df_exp, 'type', col_filter, col_id, True)
#df_exp_grouped

# Pivot result, reset index, and rename the axis.
df_exp_pivoted = df_exp_grouped.pivot(index=['type'], columns='filter', values='number')
df_exp_pivoted = df_exp_pivoted.reset_index()
df_exp_pivoted.rename_axis(columns=None, inplace=True)
#df_exp_pivoted

In [ ]:

# Perform LEFT JOIN for df_exp_grouped_type, df_exp_pivoted on 'type'.
df_exp_merged = pd.merge(df_exp_grouped_type, df_exp_pivoted, on=['type'], how='left')
#df_exp_merged

## Exposure Counts

In [ ]:

# Add a "TOTAL" row at the end of df_exp_merged to contain the summed totals
#  of each numerical column.

# Instantiate df_totals dataframe.
df_totals = pd.DataFrame({'type': ['TOTAL']})

# Get the sum of each numerical column and add it as a new row.
for col in df_exp_merged.select_dtypes(include='number'):
    df_totals[col] = [df_exp_merged[col].sum()]

df_final = pd.concat([df_exp_merged, df_totals], ignore_index=True)

df_final

## Science Exposure Counts

In [ ]:

### 3.2.2 All Science Exposures

In [ ]:

# First, group by col_sciprog, col_target, 'tract', and col_filter.
df_sci_grouped = df4Dgroupby(df_sci, col_sciprog, col_target, 'tract', col_filter, col_id, True)
#df_sci_grouped

In [ ]:

# Next, pivot the dataframe, reset the index, and rename the axis.
#  Kudos to Poe.com AI assistant.
df_sci_pivoted = df_sci_grouped.pivot(index=[col_sciprog, col_target, 'tract'], columns=col_filter, values='number')
df_sci_pivoted = df_sci_pivoted.reset_index()
df_sci_pivoted.rename_axis(columns=None, inplace=True)
#df_sci_pivoted

In [ ]:

# Next,group by col_sciprog, col_target, and 'tract'.
df_sci_grouped_tract = df3Dgroupby(df_sci, col_sciprog, col_target, 'tract', col_id, False)
df_sci_grouped_tract.rename(columns={'number': 'n_exp'}, inplace=True)
#df_sci_grouped_tract

In [ ]:

# Next, perform a LEFT JOIN for df_sci_grouped_tract, df_sci_pivoted on col_sciprog, col_target.
df_sci_merged = pd.merge(df_sci_grouped_tract, df_sci_pivoted, on=[col_sciprog, col_target, 'tract'], how='left')
#df_sci_merged

In [ ]:

# Due to dithering and/or pointing errors, some (col_sciprog, col_target) pairs
#  may have multiple matches to the closest tract to the beam sight of the exposure.
#  Here, we combine combine all the matches for each (col_sciprog, col_target) pair
#  into a single dataframe row.
# Kudos again to Poe.com AI assistant for this solution!

#  Group df_sci_merged by (col_sciprog, col_target) pair.
df_sci_merged_grp = df_sci_merged.groupby([col_sciprog, col_target])

#  Identify all columns containing numerical values; these will be summed for 
#   each (col_sciprog, col_target) pair.
numeric_cols = df_sci_merged.select_dtypes(include='number').columns.tolist()
#  But remove the 'tract' column; we treat this numerical column differently.
numeric_cols.remove('tract')

#  And now combined tracts for each each (col_sciprog, col_target) pair, 
#   creating a comma-separated string for (col_sciprog, col_target) pairs
#   that have multiple tract matches.
df_sci_combined = df_sci_merged_grp.agg({
    'tract': lambda x: ','.join(map(str, sorted(x, key=lambda y: df_sci_merged.loc[df_sci_merged['tract'] == y, 'n_exp'].values[0], reverse=True))),
    **{col: 'sum' for col in numeric_cols}
}).reset_index()

#df_sci_combined

In [ ]:

# Add a "TOTAL" row at the end of df_sci_merged to contain the summed totals
#  of each numerical column.

# Instantiate df_totals dataframe.
df_totals = pd.DataFrame({col_sciprog: ['TOTAL'], col_target: [''], 'tract': ['']})

# Get the sum of each numerical column and add it as a new row
for col in df_sci_combined.select_dtypes(include='number'):
    df_totals[col] = [df_sci_combined[col].sum()]

df_final = pd.concat([df_sci_combined, df_totals], ignore_index=True)

df_final

***The tracts listed are those that overlap the beam sight of the exposures associated with a given (`science_program`, `target`) pair.  Due to the extent of the focal plane, there may be additional tracts associated with a given exposure pointing; these are not recorded here.*** 

***That said, dithering and/or pointing errors may mean that different exposures associated with a (`science_program`, `target`) pair may be matched with different tracts.  In that case, a comma-separated list of tracts is provided in the above table for that (`science_program`, `target`) pair; the tract list is arranged in order of decreasing number of exposures associated with that tract.***


In [ ]:

## 3.3 Positions

In [ ]:

### 3.3.1 Target RA, DEC

## Science Target Exposure Sky Positions (RA, DEC)

In [ ]:

df_sci.sort_values(by='field_name', inplace=True)

plt.figure(figsize=(10, 10))

# Plot in Cartesian coordinates
if df_sci[col_target].dtype == "int64" or df_sci[col_target].dtype == "float64":
    # We create and annotate the plot one way if col_target is numerical...
    ax = sns.scatterplot(data=df_sci, x="ra", y="dec", hue=col_target, palette='viridis', s=100)
else:
    # Otherwise, if col_target is a string or similar, we create and annotate the plot this way...
    #  (This could be done better.  As it is, the spacing is very crowded.)
    ax = sns.scatterplot(data=df_sci, x="ra", y="dec", s=100)
    prev_label = 'XXXXXX'
    for i, label in enumerate(df_sci[col_target]):
        if label != prev_label:
            ax.annotate(label, (df_sci['ra'].iloc[i], df_sci['dec'].iloc[i]), fontsize=6)
            prev_label = label

# Add labels and title
plt.xlabel('RA')
plt.ylabel('DEC')
title = """RA, DEC positions of %s Observations of Different Science Program Targets on %s""" %(instrument, day_obs)
plt.title(title)
plt.grid(True)

In [ ]:

### 3.3.2 Target vs. zenith_angle

## Science Target Exposure Zenith Angles

In [ ]:

plt.figure(figsize=(10, 10))

ax = sns.scatterplot(data=df_sci, x="zenith_angle", y=col_target)

# Add labels and title
#plt.xlabel('Zenith Angle')
#plt.ylabel('Target')
title = """Zenith distances for of %s Observations of Different Science Program Targets on %s""" %(instrument, day_obs)
plt.title(title)
plt.grid(True)


In [ ]:

### 3.3.3 Target vs. MJD

## Science Target Exposure MJDs

In [ ]:

plt.figure(figsize=(10, 10))

#ax = sns.scatterplot(data=df_sci, x="mjd", y="target")
ax = sns.scatterplot(data=df_sci, x="mjd", y=col_target)


# Add labels and title
plt.xlabel('MJD')
plt.ylabel(col_target)
title = """MJDs of %s Observations of Different Science Program Targets on %s""" %(instrument, day_obs)
plt.title(title)
plt.grid(True)


In [ ]:

### 3.3.4 Target zenith angle vs. MJD

## Science Target Exposure Zenith Angles vs. MJD

In [ ]:

df_sci.sort_values(by='mjd', inplace=True)

plt.figure(figsize=(10, 10))

# Plot in Cartesian coordinates
if df_sci[col_target].dtype == "int64" or df_sci[col_target].dtype == "float64":
    # We create and annotate the plot one way if col_target is numerical...
    ax = sns.scatterplot(data=df_sci, x="mjd", y="zenith_angle", hue=col_target, palette='viridis', s=100)
else:
    # Otherwise, if col_target is a string or similar, we create and annotate the plot this way...
    #  (This could be done better.  As it is, the spacing is very crowded.)
    ax = sns.scatterplot(data=df_sci, x="mjd", y="zenith_angle", s=100)
    prev_label = 'XXXXXX'
    for i, label in enumerate(df_sci[col_target]):
        if label != prev_label:
            #mjd = df_sci['mjd'].iloc[i]
            #za = df_sci['zenith_angle'].iloc[i]
            #print(i, label, mjd, za)
            ax.annotate(label, (df_sci['mjd'].iloc[i], df_sci['zenith_angle'].iloc[i]), fontsize=6)
            prev_label = label

# Add labels and title
plt.xlabel('MJD')
plt.ylabel('Zenith Angle')
title = """Zenith Angle vs. MJD of %s Observations of Different Science Program Targets on %s""" %(instrument, day_obs)
plt.title(title)
plt.grid(True)

In [ ]:

# 4. Final cleanup

In [ ]:

# Reset the display.max_rows option to the original default
pd.reset_option("display.max_rows")

In [ ]:

# Stop automatic execution of cells before reaching the Sandbox...
#sys.exit("Notebook execution stopped here.")

raise StopExecution

In [ ]:

# 5. Sandbox